In [1]:
# Импорт библиотеки
import pandas as pd

In [2]:
# Считываем dataframe в переменную promo
promo = pd.read_csv('/Users/anastasiakoroleva/Desktop/milk_promo_sales.csv')

In [3]:
# Просматриваем часть колонок
promo.head(10)

,store_id,period_id,sales_volume
0,4186684,226,1004.60
1,5219836,226,989.00
2,4185865,226,827.45
3,4186186,226,708.40
4,4523929,226,690.75
5,4186441,226,592.55
6,27110794,226,557.45
7,5200996,226,542.50
8,4186603,226,537.85
9,4186354,226,537.65


In [4]:
# Проверяем наличие пропусков в таблице
promo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846990 entries, 0 to 846989
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   store_id      846990 non-null  int64  
 1   period_id     846990 non-null  int64  
 2   sales_volume  846990 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 19.4 MB


In [5]:
# Сортировка таблицы по столбцам 'store_id' и 'period_id'
promo.sort_values(['store_id', 'period_id'])

,store_id,period_id,sales_volume
724224,4168621,191,2.2500
216520,4168621,192,6.3500
175105,4168621,193,7.0500
699084,4168621,194,2.5000
780957,4168621,195,1.7000
...,...,...,...
512944,38126926,241,3.9500
832513,38126935,240,0.9500
417892,38126935,242,4.6500
354468,38126938,241,5.1000


In [6]:
# Переиндексация строк в отсортированной таблице и удаление старых индексов
promo = promo.sort_values(by=['store_id', 'period_id']).reset_index(drop = True)
promo

,store_id,period_id,sales_volume
0,4168621,191,2.2500
1,4168621,192,6.3500
2,4168621,193,7.0500
3,4168621,194,2.5000
4,4168621,195,1.7000
...,...,...,...
846985,38126926,241,3.9500
846986,38126935,240,0.9500
846987,38126935,242,4.6500
846988,38126938,241,5.1000


In [7]:
period_id = promo['period_id']
store_id = promo['store_id']
sales_volume = promo['sales_volume']

# Cоздаем словарь
promo = pd.DataFrame({'store_id': store_id,
                          'period_id': period_id,
                          'sales_volume': sales_volume}) 

# С помощью логического "или" объединяем два кода для заполнения promo_id
promo["promo_id"] = (
    (promo["store_id"] != promo["store_id"].shift().bfill())
    |
    (promo["period_id"].diff().fillna(1).ne(1))
).cumsum().add(1)


# Группировка по promo_id и счет индексов внутри каждой из подгрупп
promo['promo_cw'] = promo.groupby('promo_id').cumcount()
promo

,store_id,period_id,sales_volume,promo_id,promo_cw
0,4168621,191,2.2500,1,0
1,4168621,192,6.3500,1,1
2,4168621,193,7.0500,1,2
3,4168621,194,2.5000,1,3
4,4168621,195,1.7000,1,4
...,...,...,...,...,...
846985,38126926,241,3.9500,140989,0
846986,38126935,240,0.9500,140990,0
846987,38126935,242,4.6500,140991,0
846988,38126938,241,5.1000,140992,0


#### 1. Общее количество промопериодов (во всех магазинах)

In [11]:
# Общее количество промопериодов во всех магазинах равно максимальному значению в столбце promo_id
promo.promo_id.max()

140992

#### 2. Медиана продолжительности промопериода (количество недель)

In [12]:
# Медиана от максимальных значений promo_cw в каждой из подгрупп promo_id и прибавить 1, так как счет promo_cw начинается с 0 
medianduration = promo.groupby('promo_id').agg({'promo_cw': ['max']}).median() + 1
medianduration

promo_cw  max    3.0
dtype: float64

#### 3. Объем  продаж по каждому промопериоду


In [13]:
# Группировка значений по promo_id и подсчет суммы sales_volume 
salesvolume = promo.groupby('promo_id').agg({'sales_volume': ['sum']})
salesvolume

,sales_volume
,sum
promo_id,
1,86.3500
2,9.9000
3,15.8500
4,18.2000
5,48.3000
...,...
140988,4.8000
140989,3.9500


#### 4. Медиана количества промопериодов на один магазин

In [14]:
# В каждой из подгрупп store_id найдем максимальное значение promo_id
promoall = promo.groupby('store_id').agg({'promo_id': ['max']})

# Так как promo_id - это накопленная величина, для каждого следующего магазина нужно вычесть значение promo_id предыдущего
# От этой разницы найдем медианное значение 
medianquantity = (promoall - promoall.shift().fillna(0)).median()
medianquantity

promo_id  max    6.0
dtype: float64